# This Notebook reads poems from the Website "Projekt Gutenberg"

Please use it in a fair way respecting the copyrights

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle

In [2]:

index_url = 'index.html'         #should be the same on every page; navigate to                                   
                                 #../gedichte/index, not to gedichte/gedichte to get the url in the list above



In [3]:
with open("poem_url_list", "rb") as file:   
    poetry_urls = pickle.load(file)
    
task = []
for url in poetry_urls: 
    task.append([url.split('/')[3],url])

index_url = 'index.html'         #should be the same on every page; navigate to                                   
                                 #../gedichte/index, not to gedichte/gedichte to get the url in the list above    

print(task[1])   

['andersen', 'https://www.projekt-gutenberg.org/andersen/gedicht1/gedicht1.html']


In [9]:
for idx, tas in enumerate(task): 
    if tas[0] == 'schiller':
        print (url)
        print(idx)

https://www.projekt-gutenberg.org/zurlinde/gedichte/gedichte.html
192
https://www.projekt-gutenberg.org/zurlinde/gedichte/gedichte.html
193
https://www.projekt-gutenberg.org/zurlinde/gedichte/gedichte.html
194


In [33]:
title_lst = []
vers_lst = []
author_lst = []
subtitle_lst = []
ID_lst = []
cnt = 0

ID = 0
for author, URL in task:
        print(author)
        print('current ID:')
        print(ID)
        cnt +=1

        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find('ul')

        url_list = []
        if not results: 
            url_cut = len(URL.split('/')[-1])
            new_url = (URL[:-url_cut] + '.html')
            page = requests.get(new_url)
            soup = BeautifulSoup(page.content, "html.parser")
            results = soup.find('ul')
            print(new_url)
        
        if results:
            
            for result in results:
                res = result.find('a')
                url_cut = len(URL.split('/')[-1])

                if res != -1:
                    link = res['href']
                    #title = re.search("titlepage", link)
                    #if not title:
                    url_list.append(URL[:-url_cut] + link)
   
            
            
            
            for url in url_list[1:]:
                    page = requests.get(url)
                    soup = BeautifulSoup(page.content, "html.parser")
                #try:
                    if soup:
                     
                        try:
                            title = soup.find('h3').string
                        except:
                            try:
                                title = soup.find('h4').string
                            except:
                                title = None
                        try: 
                            sub_title = soup.find('p',class_='center').string
                        except:
                            sub_title = ''      

                        if not title:
                            title = soup.find('h3')

                            if not title:                
                                title = soup.find('h4')

                            if title:                        
                                title = re.sub("<.*?>",'',str(title))  

                        if title == 'Inhalt':
                            title = soup.find_all('h3')

                            if not title:                
                                title = soup.find_all('h4')
                            try:   
                                title = str(title[1])
                                title = re.sub("<.*?>",'',str(title)) 
                            except:
                                pass

                        if (soup.find_all('p',class_='vers')) :    # first html scheme used for poems
                            entered = False
                            for vers in (soup.find_all('p',class_='vers')):
                                entered = True
                                vers_lst.append(vers.text)
                                title_lst.append(title)
                                author_lst.append(author)
                                subtitle_lst.append(sub_title)
                                ID_lst.append(ID)
                            if not entered: 
                                print('error not entered')
                                print(url)

                            ID += 1
                        else:                                        # second html scheme used for poems
                            poem = soup.find('table',class_='poem')
                            if poem:
                                poem_comp_lst = poem.find_all('td')

                                for item in poem_comp_lst:

                                    if not item.text.isspace():
                                        poem_comp = item
                                        break

                                poem_p = poem_comp.find_all('p')

                                p_tags = poem_comp.find_all('p')

                                if p_tags:                                 # more then one strophe
                                    for tag in p_tags:
                                        tag.extract()
                                    title_lst.append(title)
                                    vers_lst.append(poem_comp.text)
                                    author_lst.append(author)
                                    subtitle_lst.append(sub_title)
                                    ID_lst.append(ID)

                                    for vers in p_tags:
                                        title_lst.append(title)
                                        vers_lst.append(vers.text)
                                        author_lst.append(author)
                                        subtitle_lst.append(sub_title)
                                        ID_lst.append(ID)
                                else:                                    # only one strophe
                                    title_lst.append(title)
                                    vers_lst.append(poem_comp.text)
                                    author_lst.append(author)
                                    subtitle_lst.append(sub_title)
                                    ID_lst.append(ID)





                                ID += 1  
                            else: 
                                print('not found')
                                print(url)
                        
        else:
            print('not found')
            print(URL)

            #except Exception as e: 
                #print('error inner loop')
                #print(e)
                #print(url)

        if cnt % 20 == 0:
            print('processed ' + str(cnt) + ' authors')
            vers_df = pd.DataFrame(list(zip(ID_lst,author_lst,title_lst, subtitle_lst,vers_lst)),
               columns =['ID','author','title','subtitle', 'vers'])

            vers_df.to_csv('gutenberg_'+str(cnt)+'.csv',index=False)
            


        

vers_df = pd.DataFrame(list(zip(ID_lst,author_lst,title_lst, subtitle_lst,vers_lst)),
               columns =['ID','author','title','subtitle', 'vers'])

vers_df.to_csv('gutenberg.csv',index=False)

vers_df

alexis
current ID:
0
andersen
current ID:
2
not found
https://www.projekt-gutenberg.org/andersen/gedicht1/chap001.html
andersen
current ID:
28
andersen
current ID:
53
andersen
current ID:
80
anzengru
current ID:
85
arndt
current ID:
154
arndt
current ID:
267
not found
https://www.projekt-gutenberg.org/arndt/gedicht1/chap001.html
arndt
current ID:
365
arnim
current ID:
460
arnim
current ID:
465
arnimb
current ID:
530
avenariu
current ID:
541
baggesen
current ID:
547
not found
https://www.projekt-gutenberg.org/baggesen/gedichte/chap001.html
ball
current ID:
577
baudelai
current ID:
623
not found
https://www.projekt-gutenberg.org/baudelai/gedipros/chap001.html
not found
https://www.projekt-gutenberg.org/baudelai/gedipros/chap002.html
not found
https://www.projekt-gutenberg.org/baudelai/gedipros/chap003.html
not found
https://www.projekt-gutenberg.org/baudelai/gedipros/chap004.html
not found
https://www.projekt-gutenberg.org/baudelai/gedipros/chap005.html
not found
https://www.projekt-gute

hoddis
current ID:
7239
hofmanns
current ID:
7263
hoelderl
current ID:
7319
hoelderl
current ID:
7486
hoelty
current ID:
7641
holz
current ID:
7660
horvath
current ID:
7664
not found
https://www.projekt-gutenberg.org/horvath/verse/chap009.html
not found
https://www.projekt-gutenberg.org/horvath/verse/chap010.html
huchric
current ID:
7679
not found
https://www.projekt-gutenberg.org/huchric/gedichte/chap222.html
huchric
current ID:
7900
huchric
current ID:
8083
ibsen
current ID:
8143
jacobi
current ID:
8206
jacobows
current ID:
8274
jacobsen
current ID:
8279
not found
https://www.projekt-gutenberg.org/jacobsen/ged-entw/chap008.html
not found
https://www.projekt-gutenberg.org/jacobsen/ged-entw/chap011.html
not found
https://www.projekt-gutenberg.org/jacobsen/ged-entw/chap031.html
not found
https://www.projekt-gutenberg.org/jacobsen/ged-entw/chap033.html
not found
https://www.projekt-gutenberg.org/jacobsen/ged-entw/chap034.html
jensen
current ID:
8309
kalckreu
current ID:
8313
not found
ht

schopenh
current ID:
14089
schubart
current ID:
14092
processed 200 authors
schwab
current ID:
14136
not found
https://www.projekt-gutenberg.org/schwab/gedicht1/chap001.html
not found
https://www.projekt-gutenberg.org/schwab/gedicht1/chap088.html
schwab
current ID:
14265
seidelh
current ID:
14268
seidl
current ID:
14273
seume
current ID:
14379
shelleyp
current ID:
14383
not found
https://www.projekt-gutenberg.org/shelleyp/lyrgedic/chap024.html
simrock
current ID:
14413
sorge
current ID:
14479
soyfer
current ID:
14537
not found
https://www.projekt-gutenberg.org/soyfer/misc/chap01.html
spittelr
current ID:
14539
stadler
current ID:
14560
stelzham
current ID:
14580
stieglit
current ID:
14616
storm
current ID:
14618
storm
current ID:
14867
strachwz
current ID:
15169
stramm
current ID:
15252
thoma
current ID:
15323
thuemmel
current ID:
15328
not found
https://www.projekt-gutenberg.org/thuemmel/gedicht1/chap035.html
not found
https://www.projekt-gutenberg.org/thuemmel/gedicht1/chap043.html
t

,ID,author,title,subtitle,vers
0,0,alexis,Entführung,,"O Lady Judith, spröder Schatz,\n Drückt..."
1,0,alexis,Entführung,,"Hart ist der Sitz und knapp und schmal,\n ..."
2,0,alexis,Entführung,,Sechs Nächte lag ich in Sumpf und Moor\n ...
3,1,alexis,Walpurgisnacht,,"Liebe Mutter, heut' Nacht heulte Regen und Win..."
4,1,alexis,Walpurgisnacht,,"Liebe Mutter, es donnerte auf dem Brocken drob..."
...,...,...,...,...,...
105985,17004,zurlinde,Der Mensch,,Das ist Seele. Und nun schaun\n Wir nich...
105986,17004,zurlinde,Der Mensch,,Der ist TREUE. Aus Unendlich\n Sehen wir...
105987,17004,zurlinde,Der Mensch,,"Das ist WUERDE. So bescheiden –\n WERT, ..."
105988,17004,zurlinde,Der Mensch,,Aus dem WAHN. Und Allerseelen\n Weiß mei...


In [31]:
vers_df = pd.DataFrame(list(zip(ID_lst,author_lst,title_lst, vers_lst)),
               columns =['ID','author','title','vers'])

In [32]:
vers_df

,ID,author,title,vers
0,0,alexis,Entführung,"O Lady Judith, spröder Schatz,\n Drückt..."
1,0,alexis,Entführung,"Hart ist der Sitz und knapp und schmal,\n ..."
2,0,alexis,Entführung,Sechs Nächte lag ich in Sumpf und Moor\n ...
3,1,alexis,Walpurgisnacht,"Liebe Mutter, heut' Nacht heulte Regen und Win..."
4,1,alexis,Walpurgisnacht,"Liebe Mutter, es donnerte auf dem Brocken drob..."
...,...,...,...,...
105985,17004,zurlinde,Der Mensch,Das ist Seele. Und nun schaun\n Wir nich...
105986,17004,zurlinde,Der Mensch,Der ist TREUE. Aus Unendlich\n Sehen wir...
105987,17004,zurlinde,Der Mensch,"Das ist WUERDE. So bescheiden –\n WERT, ..."
105988,17004,zurlinde,Der Mensch,Aus dem WAHN. Und Allerseelen\n Weiß mei...
